<a href="https://colab.research.google.com/github/pablo-sampaio/espacos4.0_pe/blob/main/IA_Python_03_AprendizagemReforco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizagem de Máquina - Parte 3

Neste notebook, mostramos rapidamente como usar **aprendizagem por reforço** para resolver uma tarefa simples. 

> Vamos usar o algoritmo **PPO** (do módulo chamado *stable-baselines*) 

> O ambiente é chamado de **LunarLander-v2** e ele simula a tarefa de pousar um "módulo lunar" (oferecido pelo módylo *gym*).

## 1 - Instalação de pacotes 

> Atenção: execute 1 vez esta seção e, depois, reinicie o ambiente (menu `Ambiente de Execução` $\rightarrow$ `Reiniciar ambiente de execução`).

In [ ]:
from IPython.display import clear_output
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install gym[all]==00.25.1
!pip install pyglet
!pip install stable-baselines3[extra]==1.6.0
!pip install importlib-metadata==4.13

#clear_output()

## 2 - Definições e Importações

> Depois de executar a seção 1 e reiniciar o ambiente, você pode executar daqui para baixo!


### 2.1 Funções Auxiliares

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

### 2.2 Imports

In [ ]:
import gym
import tensorboard

%load_ext tensorboard

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.ppo import MlpPolicy

## 3 - Cria o ambiente e instancia o agente

Abaixo, criamos o ambiente da tarefa **LunarLander-v2** .

In [ ]:
# Diferença em relação ao vídeo: criei uma variável para guardar o nome do ambiente
# Nomes de ambientes para testar: "CartPole-v1", "BipedalWalker-v3", "MountainCar-v0", "LunarLander-v2"
# Veja mais em https://www.gymlibrary.dev/

NOME_AMBIENTE = "LunarLander-v2"

ambiente = gym.make(NOME_AMBIENTE)


A seguir, criamos o modelo (ou agente) do tipo **PPO**, passando este dois argumentos principais:
- o ambiente que o agente irá executar no seu aprendizado 
- *MlpPolicy* para fazer o agente usar uma rede neural MLP internamente.

In [ ]:
modelo = PPO(MlpPolicy, ambiente, tensorboard_log="log_dir", verbose=1)

Vamos deixar o modelo/agente interagir com esse ambiente antes do treinamento, para comparar depois:

In [ ]:
recompensa_media, _ = evaluate_policy(modelo, ambiente, n_eval_episodes=30, deterministic=False)

In [ ]:
print(f"Recompensa média: {recompensa_media:.2f}")

E agora vamos gravar um vídeo para vê-lo em ação, antes de treinado.

In [ ]:
record_video(NOME_AMBIENTE, modelo, video_length=1000, prefix='ppo-sem-treino')
show_videos('videos', prefix='ppo-sem-treino')

## 4 - Ativa o Tensorboard

Aqui, vamos ativar um recurso para acompanhar o treinamento.

In [ ]:
%tensorboard --logdir log_dir

## 5 - Treina o agente

Abaixo, rodamos a função de treinamento. Ela irá executar a simulação várias vezes (por vários *episódios*) até completar 150 mil passos de simulação (150 mil ações executadas pelo agente).

In [ ]:
modelo.learn(total_timesteps=150000)

Depois, gravamos o vídeo do agente treinado.

In [ ]:
record_video(NOME_AMBIENTE, modelo, video_length=4000, prefix='ppo-treinado')
show_videos('videos', prefix='ppo-treinado')

Por fim, vamos fazer o agente treinado executar ações no ambiente. Compare com o desempenho antes de treinar.

In [ ]:
recompensa_media, _ = evaluate_policy(modelo, ambiente, deterministic=False, n_eval_episodes=30)

In [ ]:
print(f"Recompensa média: {recompensa_media:.2f}")